# Single A2A Example

This notebook demonstrates how to run a single Allocation-to-Allocators (A2A) job from the AlloOptim library.

## 1. Import Libraries

Import the necessary libraries for A2A optimization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# AlloOptim imports
from allo_optim.allocation_to_allocators.allocation_orchestrator import (
    AllocationOrchestrator,
    AllocationOrchestratorConfig,
    OrchestrationType,
)
from allo_optim.config.stock_universe import list_of_dax_stocks

print("Libraries imported successfully!")

## 2. Create Sample Portfolio Data

Create sample price data for a portfolio of assets.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Get sample stocks (using first 5 DAX stocks)
all_stocks = list_of_dax_stocks()[:5]
assets = [stock.symbol for stock in all_stocks]

# Generate 6 months of daily price data
dates = pd.date_range("2023-01-01", periods=126, freq="D")
n_assets = len(assets)

# Generate correlated price movements
returns = np.random.multivariate_normal(
    mean=np.full(n_assets, 0.0005),  # 0.05% daily return
    cov=np.eye(n_assets) * 0.0004 + np.full((n_assets, n_assets), 0.0001),  # Some correlation
    size=len(dates),
)

# Create price DataFrame
prices = pd.DataFrame(
    100 * np.exp(np.cumsum(returns, axis=0)),
    index=dates,
    columns=assets
)

print("Sample Data Created:")
print(f"Assets: {', '.join(assets)}")
print(f"Date range: {dates[0].date()} to {dates[-1].date()}")
print(f"Price data shape: {prices.shape}")
print(f"\nSample prices (first 3 rows):")
print(prices.head(3))

## 3. Run A2A Optimization

Initialize and run the Allocation-to-Allocators optimization with equal weighting.

In [ ]:
# Configure A2A with 3 optimizers
optimizer_names = ["MaxSharpe", "RiskParity", "Naive"]

# Create configuration for equal weighting
config = AllocationOrchestratorConfig(
    orchestration_type=OrchestrationType.EQUAL,
)

# Create orchestrator
orchestrator = AllocationOrchestrator(
    optimizer_names=optimizer_names,
    config=config,
)

print(f"A2A Configuration:")
print(f"Optimizers: {', '.join(optimizer_names)}")
print(f"Orchestration: {config.orchestration_type.value}")

# Run A2A optimization
time_today = datetime.now()
result = orchestrator.allocate(
    all_stocks=all_stocks,
    time_today=time_today,
    df_prices=prices,
)

print(f"\nA2A Result:")
print(f"Success: {result.success}")
if result.success:
    print("Optimization completed successfully!")
else:
    print(f"Error: {result.error_message}")

## 4. Display Results

Show the final portfolio allocation and optimizer weights.

In [ ]:
if result.success:
    # Display asset weights
    print("Final Portfolio Allocation:")
    print("-" * 40)
    sorted_weights = sorted(result.asset_weights.items(), key=lambda x: x[1], reverse=True)
    for asset, weight in sorted_weights:
        print(f"{asset}: {weight:.4f} ({weight*100:.2f}%)")

    total_weight = sum(result.asset_weights.values())
    print(f"\nTotal weight: {total_weight:.6f}")

    # Display optimizer weights
    if hasattr(result.statistics, 'algo_weights'):
        print(f"\nOptimizer Weights:")
        print("-" * 40)
        for optimizer, weight in result.statistics.algo_weights.items():
            print(f"{optimizer}: {weight:.4f} ({weight*100:.2f}%)")

    # Create visualization
    plt.figure(figsize=(12, 5))

    # Asset allocation bar chart
    plt.subplot(1, 2, 1)
    assets_list = [asset for asset, _ in sorted_weights]
    weights_list = [weight for _, weight in sorted_weights]
    plt.bar(assets_list, weights_list, color='skyblue')
    plt.title('Asset Allocation')
    plt.xlabel('Assets')
    plt.ylabel('Weight')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

    # Optimizer weights bar chart
    plt.subplot(1, 2, 2)
    if hasattr(result.statistics, 'algo_weights'):
        optimizers = list(result.statistics.algo_weights.keys())
        opt_weights = list(result.statistics.algo_weights.values())
        plt.bar(optimizers, opt_weights, color='lightgreen')
        plt.title('Optimizer Weights')
        plt.xlabel('Optimizers')
        plt.ylabel('Weight')
        plt.xticks(rotation=45)
        plt.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("✅ A2A optimization completed successfully!")

else:
    print("❌ A2A optimization failed")
    if hasattr(result, 'error_message'):
        print(f"Error: {result.error_message}")